# Tutorial on manipulating OLCI L1 EFR product

In [ ]:
from pathlib import Path
import xarray as xr
from eopf import EOProduct
from eopf.common.constants import OpeningMode

In [ ]:
SAMPLE_PATH = Path("/mount/internal/work-st/projects/cs-412/2078-dpr/Samples/Products/Zarr_Beta")
files = [ f for f in SAMPLE_PATH.glob("S3OLCEFR*.zarr")]
product = files[0]
product

### Open the product using EOPF API and visualize the tree structure
Note: there are still warning messages due to problems of dependencies in the CPM package

In [ ]:
prod=EOProduct("prod",url=str(product)).open(mode=OpeningMode.OPEN)
prod


Plotting data fails with version 1.5.1

In [ ]:
prod.measurements.image.oa01_radiance.plot()

### Opening gridded measurement data data with xarray

In [ ]:
group = "measurements/image"
# Open with xarray zarr engine
ds=xr.open_dataset(SAMPLE_PATH / product / group, engine="zarr",chunks={})
# Open with custom engine="eop"
ds=xr.open_dataset(SAMPLE_PATH / product, group=group, engine="eop")
rad = ds.oa01_radiance
rad

Underlying data is dask.array

In [ ]:
rad.data

#### Simple raster plot
Note that using xarray, data is correctly decoded and masked

In [ ]:
rad.plot()

#### Plot using the coordinates (lon,lat)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14, 6))
ax = plt.axes()
ds.oa01_radiance.plot.pcolormesh(
    ax=ax,
    x="longitude", y="latitude", add_colorbar=False
)

#### Open Meteorological conditions

In [ ]:
group = "conditions/meteo"
ds=xr.open_dataset(SAMPLE_PATH / product / group, engine="zarr")
ds

#### Interpolate the atmo. temp. profile at p=832.2 hPa and plot

In [ ]:
tp=ds["atmospheric_temperature_profile"].interp(tie_pressure_levels=832.2)
tp.plot()

#### Open sat/sun angles
Angles are stored on a tiepoint subgrid

In [ ]:
group = "conditions/geometry"
ds=xr.open_dataset(SAMPLE_PATH / product / group, engine="zarr",chunks={})
ds

#### Have a look at condition parameters for removed pixels

In [ ]:
group="conditions/orphans"
ds=xr.open_dataset(SAMPLE_PATH / product / group, engine="zarr",chunks={})
ds